This code allows to scrap data from different subreddit, and keep the dataset up to date. It can be used in any subreddit.

In [2]:
import pandas as pd
import numpy as np
import requests
import datetime
import time
import json
print('Libs imported')

Libs imported


In [3]:
def get_submission_from_utc(subreddit,created_utc):
    url_sub = 'https://api.pushshift.io/reddit/search/submission/?size=1000&after='+str(created_utc)+'&subreddit='+str(subreddit)
    r_sub = requests.get(url_sub)
    data_sub = json.loads(r_sub.text)
    return data_sub

def get_submission_before_utc(subreddit,created_utc):
    url_sub = 'https://api.pushshift.io/reddit/search/submission/?size=1000&before='+str(created_utc)+'&subreddit='+str(subreddit)
    r_sub = requests.get(url_sub)
    data_sub = json.loads(r_sub.text)
    return data_sub

def create_internal_dic(submission):
    error_count=0
    com_list=[]
    try:
        if '[removed]'in submission['selftext']:
            internal_dict=0
        else:
            try:
                sub_id=submission['id']
                url_com='https://api.pushshift.io/reddit/comment/search/?link_id='+str(sub_id)+'&limit=20000'
                data_com=json.loads(requests.get(url_com).text)
                for com in data_com['data']:
                    com_list.append(com['body'])
                internal_dict={'id':submission['id'],'created_utc':submission['created_utc'],'body':submission['selftext'],'commentaries':com_list}
            except:
                error_count=error_count+1
                internal_dict=0
    except:
        internal_dict=0
        error_count=error_count+1
    return {'dic':internal_dict, 'errors':error_count}

In [5]:
try:
    with open("sub_scrapping.json","r") as data:
        main_json=json.load(data)
        data.close()
        print('File already exist')

except:
    
    with open("sub_scrapping.json","w") as data:
        subreddit_list=[]
        print('Enter the name of the subreddit to get the data from:')
        subreddit=input()

        while subreddit!='':
            print('If you want to scrap another subreddit you can write its name. Else press enter')
            subreddit_list.append(subreddit)
            subreddit=input()

        day_stamp=int(input('How many days of data do you want to gather for these subreddits'))

        data.write('{"data":{')
        for subreddit in subreddit_list:
            ini=False
            count=0
            total_errors=0
            ini_number=0
            print('Data gathering for '+str(subreddit))
            data.write('"'+str(subreddit)+'":[')
            today_date = datetime.datetime.today()
            start_date = today_date - datetime.timedelta(days=day_stamp)
            scrap_date=int(start_date.timestamp())
            data_sub=get_submission_from_utc(subreddit,scrap_date)
            while not ini:
                ini_number=ini_number+1
                dic=create_internal_dic(data_sub['data'][ini_number-1])
                if dic['dic']==0:
                    pass
                else:
                    data.write(json.dumps(dic['dic']))
                    ini=True
            for sub in data_sub['data'][ini:]:
                dic=create_internal_dic(sub)
                if dic['dic']==0:
                    pass
                else:
                    data.write(','+json.dumps(dic['dic']))
                total_errors=total_errors+dic['errors']

            data_sub=get_submission_from_utc(subreddit,data_sub['data'][len(data_sub['data'])-1]['created_utc'])    
            while len(data_sub['data'])!=0:
                count=count+1

                for sub in data_sub['data']:
                    dic=create_internal_dic(sub)
                    if dic['dic']==0:
                        pass
                    else:
                        data.write(','+json.dumps(dic['dic']))
                    total_errors=total_errors+dic['errors']            

                print('\t'+str(count)+' Data scrapped to the '+time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(data_sub['data'][len(data_sub['data'])-1]['created_utc']))+' with a total of '+str(total_errors)+' errors')
                data_sub=get_submission_from_utc(subreddit,data_sub['data'][len(data_sub['data'])-1]['created_utc'])
            if subreddit_list.index(subreddit)==len(subreddit_list)-1:
                data.write(']}')
            else:
                data.write('],')
        data.write('\n}')

Enter the name of the subreddit to get the data from:
investing
If you want to scrap another subreddit you can write its name. Else press enter

How many days of data do you want to gather for these subreddits2
Data gathering for investing


In few words, the code above creates a json file containing every submission and commentaries from one or more subreddit. It serves as a database initialization.


The structure of the JSON will be :
{
    data:
         {subreddit1:
                    [{'id':str,'created_utc':int,'body':str,'commentaries':list]
         ,
         subreddit2:
                    [{}]
         }
}
                        

Let's breakdown the code piece by piece. The initialization is like that: 
1. 

```python
try:
    with open("sub_scrapping.json","r") as data:
        main_json=json.load(data)
        data.close()
        print('File already exist')

except:
    
    with open("sub_scrapping.json","w") as data:
        subreddit_list=[]
        print('Enter the name of the subreddit to get the data from:')
        subreddit=input()

        while subreddit!='':
            print('If you want to scrap another subreddit you can write its name. Else press enter')
            subreddit_list.append(subreddit)
            subreddit=input()

        day_stamp=int(input('How many days of data do you want to gather for these subreddits'))

        data.write('{\n\t"data":')
```
This part create a file that will be then used as a data base. As it creates a file, it can delete it, so it is important to check first if the file exists, this is the role of the try except. If the JSON file is loaded without a problem, the script ends.

Otherwise, the user has to input 2 informations:
- the subreddits he wants to scrap
- the number of days he wants to scrap

As it can be a very time consuming script, it is better to initialize with a small amount of days, and then use the next part of the program, which is dedicated to modify the database

2.

Next, the script will iterate through the list of subreddit the user wrote: 

```python
        for subreddit in subreddit_list:
            ini=False
            count=0
            total_errors=0
            ini_number=0
            print('Data gathering for '+str(subreddit))
            data.write('\n\t\t{\n\t\t\t"'+str(subreddit)+'": \n\t\t\t\t[')
            today_date = datetime.datetime.today()
            start_date = today_date - datetime.timedelta(days=day_stamp)
            scrap_date=int(start_date.timestamp())
            
```
Before anything, it is important to know that the way data are written in the file is a bit different than expected. What could be expected, is that once the dictionnary is built the data is then written in the file. However I had a lot of internet issues, and needed a way to keep the data even if the programm crashed due to internet issues.
Therefore, each time the programm iterates, it writes data in the file, not in the end. To make it simple, instead of writing the entire file at once, it writes it line by line.

ini=False is to make a special case of the first "submission" treated. Because data is getting written item by item, a coma is added before one, so when it is loaded as a JSON file there are major issues.

count=0 is to show the number of iteration each command needs. It is reseted for each subreddit.
total_errors is self explanatory, it keep tracks of the number of time the algorithm didn't manage to scrap the submissions.

For the file creation, the logic is the following:
The user wants to scrap x days, so the program substracts x days to today and the script scrap the data from the scrap date (epoch format)

3.

Then the data gathering happens.

```python
data_sub=get_submission_from_utc(subreddit,scrap_date)
            while not ini:
                ini_number=ini_number+1
                dic=create_internal_dic(data_sub['data'][ini_number-1])
                if dic['dic']==0:
                    pass
                else:
                    data.write(json.dumps(dic['dic']))
                    ini=True
            for sub in data_sub['data'][ini:]:
                dic=create_internal_dic(sub)
                if dic['dic']==0:
                    pass
                else:
                    data.write(','+json.dumps(dic['dic']))
                total_errors=total_errors+dic['errors']
```

The function: get_function_from_utc() is an api calls to pushift.io, a usefull tool to scrap reddit, the function returns 100 submissions maximum.

Then, the create_internal_dic() is here to create a dictionnary including the id, created_utc, body and commentaries of one submission. It returns the dic plus the number of errors encountered.

There is a specific condition for the very first item, as the dic is dumped without coma, then the entire data gathering takes place.

The one important thing to notice is that the scrap date is replaced every iteration by the most recent dates. 
The iteration process stops when no more submissions are returned ie. when the length of the list equals 0

The second piece of code is to manage the data file:
There are 3 different actions possible:
- Updating the database (UP) will get the most recent date (highest epoch) and scrap every subreddits already in the file until today's date
- Get more data will allow the user to collect more data, anterior to the oldest date (lowest epoch) in the dataset. 
- Add a subreddit, will append another subreddit to the already existing subreddit list

In [ ]:
count=0
total_errors=0
with open("sub_scrapping.json","r") as data:
    main_json=json.load(data)
    data.close()
existing_sub=list(main_json['data'].keys())
action=input('Subreddits already scrapped: '+str(existing_sub)+'\nUpdate the list, Add another subreddit or Get more data (UP/ADD/GET)?')
if str(action).upper() =='UP':
    for subreddit in existing_sub:
        print("Number of submissions before update in "+str(subreddit)+": "+str(len(main_json['data'][subreddit])))
        count=0
        max_date=main_json['data'][subreddit][0]['created_utc']
        for submission in main_json['data'][subreddit]:
            if submission['created_utc']>max_date:
                max_date=submission['created_utc']
            else:
                pass
        print('Last date detected for '+str(subreddit)+': '+time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(max_date)))
        print('Data updating')
        data_sub=get_submission_from_utc(subreddit,max_date)
        while len(data_sub['data'])!=0:
            count=count+1
            for sub in data_sub['data']:
                dic=create_internal_dic(sub)
                if dic['dic']==0:
                    print(sub['selftext'])
                else:
                    main_json['data'][subreddit].append(dic['dic'])
            max_date=data_sub['data'][len(data_sub['data'])-1]['created_utc']
            total_errors=total_errors+dic['errors']  
            data_sub=get_submission_from_utc(subreddit,max_date)
            print('\t'+str(count)+' Data updated to the '+time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(max_date))+' with a total of '+str(total_errors)+' errors')
        print("Number of submissions after update in "+str(subreddit)+": "+str(len(main_json['data'][subreddit])))

elif str(action).upper()=='GET':
    date_to_add=input('How many days do you want to add ?')
    for subreddit in existing_sub:
        print("Number of submissions before data gathering in "+str(subreddit)+": "+str(len(main_json['data'][subreddit])))
        count=0
        min_date=main_json['data'][subreddit][0]['created_utc']
        for submission in main_json['data'][subreddit]:
            if submission['created_utc']<min_date:
                min_date=submission['created_utc']
            else:
                pass
        print('Last date detected for '+str(subreddit)+': '+time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(min_date)))
        print('Data updating')
        goal_date = datetime.datetime.fromtimestamp(min_date) - datetime.timedelta(days=int(date_to_add))
        goal_epoch=int(goal_date.timestamp())
        while min_date>goal_epoch:
            data_sub=get_submission_before_utc(subreddit,min_date)
            count=count+1
            for sub in data_sub['data']:
                dic=create_internal_dic(sub)
                if dic['dic']==0:
                    pass
                else:
                    main_json['data'][subreddit].append(dic['dic'])
            min_date=data_sub['data'][len(data_sub['data'])-1]['created_utc']
            total_errors=total_errors+dic['errors']  
            print('\t'+str(count)+' Data updated to the '+time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(min_date))+' with a total of '+str(total_errors)+' errors')
        print("Number of submissions after data gathering in  "+str(subreddit)+": "+str(len(main_json['data'][subreddit])))

elif str(action).upper()=='ADD':
    sub_to_add=input('Which subreddit do you want to add ?')
    limit_date=input('How many days do you want to scrap ?')
    goal_date = datetime.datetime.today() - datetime.timedelta(days=int(limit_date))
    goal_epoch=int(goal_date.timestamp())
    scrap_date=int(datetime.datetime.today().timestamp())
    main_json['data'][sub_to_add]=[]
    while scrap_date>goal_epoch:
        data_sub=get_submission_before_utc(sub_to_add,scrap_date)
        count=count+1
        for sub in data_sub['data']:
            dic=create_internal_dic(sub)
            if dic['dic']==0:
                pass
            else:
                 main_json['data'][sub_to_add].append(dic['dic'])
        scrap_date=data_sub['data'][len(data_sub['data'])-1]['created_utc']
        total_errors=total_errors+dic['errors']  
        print('\t'+str(count)+' Data added to the '+time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(scrap_date))+' with a total of '+str(total_errors)+' errors')
    print("Number of submissions written in "+str(sub_to_add)+": "+str(len(main_json['data'][sub_to_add])))
    
    
with open("sub_scrapping.json","w") as data:
    json_file=json.dumps(main_json)
    data.write(json_file)
    data.close()         

Subreddits already scrapped: ['pennystocks', 'investing', 'wallstreetbets']
Update the list, Add another subreddit or Get more data (UP/ADD/GET)?get
How many days do you want to add ?2
Nombre d'éléments avant addition des données dans pennystocks: 954
Last date detected for pennystocks: 2021-03-12 02:26:01
Data updating
	1 Data updated to the 2021-03-11 21:30:39 with a total of 0 errors
	2 Data updated to the 2021-03-11 18:48:25 with a total of 0 errors
	3 Data updated to the 2021-03-11 15:51:23 with a total of 0 errors
	4 Data updated to the 2021-03-11 05:45:02 with a total of 0 errors
	5 Data updated to the 2021-03-10 21:52:50 with a total of 0 errors
	6 Data updated to the 2021-03-10 17:54:53 with a total of 0 errors
	7 Data updated to the 2021-03-10 14:01:39 with a total of 0 errors
	8 Data updated to the 2021-03-10 02:34:09 with a total of 0 errors
	9 Data updated to the 2021-03-09 20:36:55 with a total of 0 errors
Nombre d'éléments après addition des données pennystocks: 1460
Nom